In [65]:
from scipy import integrate
import matplotlib.pyplot as plt
import os
import numpy as np
from matplotlib import pyplot as plt      
import pandas as pd
from scipy import special                 
import array
import scipy as sp
import scipy.interpolate
import re
import pickle as pickle
def unPackRawFile(raw_path):
    """


    
    - unpacks the .raw file. Not used for the neural network.
    """
    y = np.loadtxt(raw_path, skiprows=6)
    distance = y[:,0]
    pec_vel = y[:,1]
    temp = y[:,2]
    HI_density = y[:,3]
    gas_density = y[:,4]
    gas_metallicity = y[:,5]
   
    return distance, pec_vel, temp, HI_density, gas_density, gas_metallicity

def getPos(path_LOS,linenumber=8):
    """
    the start position of the LOS is given inside each file, (in the comments)
    this function parses the comments to get that information
    """
    f = open(path_LOS)
    x = f.readlines()[linenumber]
    answer = re.search('\(([^)]+)', x).group(1)
    arr = np.array(answer.split(','),dtype=float)
    return arr

def getDir(path_LOS,linenumber=8):
    """
    the direction of the LOS is given inside each file, (in the comments)
    this function parses the comments to get that information
    """
    f = open(path_LOS)
    x = f.readlines()[linenumber]
    answer = re.search('\(([^)]+)', x.split(', ')[1]).group(1)
    arr = np.array(answer.split(','),dtype=float)
    return arr


def convertSphereToCart(theta, phi):
    "converts a unit vector in spherical to cartesian, needed for getGalaxies"
    return np.array([np.sin(theta)*np.cos(phi), np.sin(theta)*np.sin(phi), np.cos(theta)])

def calculate_gamma(temperature,density,xHI,scale):
    '''Gamma = R(T)*ne/xHI, R(T) = 4.3e-13 * (T/1e4K)^{-0.7), ne = 0.82*nb.
    '''
    omega_b = 0.0479; h = 0.6814
    mean_nb = 1.123e-5/scale**3*omega_b*h**2
    nb = np.multiply(density ,mean_nb); ne = 0.82*nb
    R_T = 4.3e-13 * (np.divide(temperature,1e4))**(-0.7)
    gamma = R_T * ne / xHI
    
def lyman_identification():
    """
    function for identifying lls using neutral fraction as filter
    """
    neutral_fraction_limit = 1e-3
    
    neutral_fraction = []
    real_lyman = []
    gamma = []
    HI = []
    gas_den = []
    real_area_under = []
    which = 0
    for i in range(51):
        raw_path = 'los.00' + '{0:03}'.format(i) +'.raw'
        distance, pec_vel, temp, HI_density, gas_density, gas_metallicity = unPackRawFile(raw_path)
        #changing the distance from chimp to cmeters
        distance_cm = distance *0.1453*3.086e24 /0.68
        position = getPos(raw_path)
        direction = convertSphereToCart(getDir(raw_path)[0],getDir(raw_path)[1])
        print(position[0]/512*40, position[1]/512*40,position[2]/512*40)
        print(direction)
        for k in range(len(distance) -2):
            if(HI_density[k]/gas_density[k]<neutral_fraction_limit and HI_density[k+1]/gas_density[k+1]>neutral_fraction_limit):
                left_bases = k+1
                peak_fraction = HI_density[k]/gas_density[k]
                peak_HI = HI_density[k]
                peak_distance = distance[k]
                peak_gas = gas_density[k]
                if( k+1 <len(distance)):
                    while(not(HI_density[k]/gas_density[k]>neutral_fraction_limit and HI_density[k+1]/gas_density[k+1]<neutral_fraction_limit) and k <len(distance)-2):
                        right_bases = k +1
                        if(k<len(distance)-1):
                            k+=1
                        if(HI_density[k]/gas_density[k]>peak_fraction):
                            peak_fraction = HI_density[k]/gas_density[k]
                            peak_HI = HI_density[k]
                            peak_distance = distance[k]
                            peak_gas = gas_density[k]
                            peak_gamma = calculate_gamma(temp[k],gas_density[k],peak_fraction,0.1452)
                    area_under = peak_HI *1/2*(distance_cm[right_bases]-distance_cm[left_bases])/0.937415
                    real_distance = [(position[0]/256*40+peak_distance*direction[0])%40, (position[1]/256*40+peak_distance*direction[1])%40,(position[2]/256*40+peak_distance*direction[2])%40]
                    print(real_distance)
                    distance_graph = []
                    HI_graph = [ ]
                    distance_real = []
                    for z in range(len(distance)):
                        if(distance[left_bases]<=distance[z]<= distance[right_bases]):
                            distance_graph.append(distance_cm[z])
                            HI_graph.append(HI_density[z])
                            distance_real.append(distance[z])
                    y_int = integrate.cumtrapz(HI_graph,distance_graph,initial = 0)
                    area_under = peak_HI *1/2*(distance_cm[right_bases]-distance_cm[left_bases])/0.937415
                    if(y_int[-1] >= (10**16)and y_int[-1]  <= (10**22) and 1.94<peak_distance<2.08):
                        real_lyman.append([y_int,real_distance,i])
                        real_area_under.append(y_int)
                        neutral_fraction.append(peak_fraction)
                        HI.append(peak_HI)
                        gamma.append(peak_gamma)
                        gas_den.append(peak_gas)
                        which = [i,distance[left_bases], distance[right_bases], peak_distance]
    return real_lyman,neutral_fraction,gamma,HI,gas_den,which,real_area_under

In [66]:
import numpy as np
real_lyman,neutral_fraction,gamma,HI,gas_den,which,real_area_under  = lyman_identification()

10.260156250000001 17.85125 10.904765625
[0.90854472 0.26859326 0.32000648]
[22.336502486377697, 36.2394206132576, 22.449227403295044]
3.2486171875 5.0693125000000006 14.574296874999998
[-0.72362636 -0.68210924 -0.105318  ]
[5.036818890200234, 8.761998961901194, 28.936042017272534]
16.503046875 1.2425 17.32484375
[-0.8921208  -0.45075957 -0.0305989 ]
[31.232798473920734, 1.5890116857935337, 34.588865155911506]
6.45740625 7.097781250000001 7.220664062499999
[-0.98377725  0.17839011  0.01895519]
[10.933819611432869, 14.554779530148318, 14.479497435943143]
14.962421875 9.426171875 12.09453125
[0.90164668 0.43198752 0.02049489]
[31.723836263204866, 19.714258207936485, 24.22995452285152]
15.153984375 10.778515625 9.066171875
[-0.91755718  0.39756977  0.0052063 ]
[28.46437617168564, 22.355844342201237, 18.142804462791496]
14.036796875 13.355390625000002 7.4311328125
[ 0.55673021 -0.76581304 -0.32184135]
[29.062129481276504, 25.350995952723405, 14.290800905704032]
[29.07056394398444, 25.33939

In [67]:
# for i in range(0,51):
#     raw_path = 'los.00' + '{0:03}'.format(i) +'.raw'
#     distance, pec_vel, temp, HI_density, gas_density, gas_metallicity = unPackRawFile(raw_path)
#     plt.plot(distance,HI_density)
#     plt.xlim(1.8,2.2)
#     plt.show()

In [68]:
for i in real_lyman:
    print(i[0][-1],i[2])

4.216735551026357e+19 0
1.187505116545934e+21 1
5.606123251815265e+19 2
3.794035913417082e+20 3
1.548152819480779e+20 4
1.592565333868983e+19 5
4.1188645255065485e+19 6
2.592124789690808e+19 7
3.10689908811999e+19 8
6.587293364828068e+18 9
1.7997153643578722e+19 10
5.355434447433235e+19 11
4.4844063681049715e+19 12
9.913985370306465e+19 13
1.1381818126349341e+18 13
3.6807886613374894e+19 14
1.197899270783723e+16 15
5.946700403718091e+18 16
1.7287741811658778e+17 17
5.485743454786878e+16 17
1.9408139609298308e+16 17
2.6723132852949398e+17 18
9.227842802690253e+16 19
2.531621618791473e+17 20
9.671882490905351e+19 21
4.9464082717968064e+17 22
1.6484361607288336e+16 23
4.1665084420771773e+18 24
7.725471017404734e+20 26
2.770565618030569e+18 27
6.538250412477896e+19 28
2.10343443837452e+20 29
8.954379245004728e+19 30
4.933567230250704e+19 31
3.763031189193534e+17 32
5.0001888310783976e+16 33
8.583146904614386e+16 34
5.736234789944569e+17 35
3.044165619920331e+19 36
3.716708606025586e+17 37
